In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import os.path
import time
import re

In [2]:
#get_ipython().system('pip3 install html5lib --user ')

#### Start you code here, you are free to add any number of cells
start=time.time()
start_day=pd.to_datetime('2009-01-01')
end_day=pd.to_datetime('2018-10-28')

url = 'https://www.estesparkweather.net/archive_reports.php?date={}'
#dates = pd.date_range('2009-01-01', '2018-10-28', freq='M')  #Real test range
dates = pd.period_range(start_day, end_day, freq='M') #test date range
final_df = pd.DataFrame(columns=['Average temperature (°F)', 'Average humidity (%)',
                             'Average dewpoint (°F)', 'Average barometer (in)',
                             'Average windspeed (mph)', 'Average gustspeed (mph)',
                             'Average direction (°deg)', 'Rainfall for month (in)',
                             'Rainfall for year (in)', 'Maximum rain per minute',
                             'Maximum temperature (°F)', 'Minimum temperature (°F)',
                             'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
                             'Minimum pressure', 'Maximum windspeed (mph)',
                             'Maximum gust speed (mph)', 'Maximum heat index (°F)', 'Date'])

In [3]:
for i in dates:
    if i.month < 10:
        date = str(i.year) + str(0) + str(i.month)
    else:
        date = str(i.year) + str(i.month)
    #print(date)
    link = url.format(date)
    data=requests.get(link)
    d=data.text
    soup=bs(d, features="html.parser")
    tables=soup.find_all("table")
    #elem=table.find_all("")
    month=[]
    for k in range(len(tables)):
        elem=tables[k].find_all("td")
        letter=elem[0].text
        if "Average" == letter[:7]:
            break
        else:
            date_str = re.search("[A-Z][a-z]+ [0-9]+", elem[0].text).group() + " " + str(i.year)
            dt = pd.to_datetime(date_str)
            #print(dt)
            if int(i.month)==int(end_day.strftime("%m")) and (int(dt.strftime('%d'))==(int(end_day.strftime('%d')) + 1)) and (int(i.year)==int(end_day.strftime("%Y"))):
                break
            else:
                elem=elem[1:]
                day=[]
                for j in range(1,len(elem),2):
                    day.append(elem[j].text)
                day.append(dt)
                month.append(day)
    df=pd.DataFrame(month, columns=['Average temperature (°F)', 'Average humidity (%)',
                             'Average dewpoint (°F)', 'Average barometer (in)',
                             'Average windspeed (mph)', 'Average gustspeed (mph)',
                             'Average direction (°deg)', 'Rainfall for month (in)',
                             'Rainfall for year (in)', 'Maximum rain per minute',
                             'Maximum temperature (°F)', 'Minimum temperature (°F)',
                             'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
                             'Minimum pressure', 'Maximum windspeed (mph)',
                             'Maximum gust speed (mph)', 'Maximum heat index (°F)', 'Date'])
    # print(df.info())
    final_df=final_df.append(df, ignore_index=True)
final_df.set_index("Date", inplace=True)
print(final_df.info())
print(final_df.head())
print(final_df.tail())
if os.path.exists("Weather_Data.csv"):
    #file=pd.read_csv("BMovies_2K.csv")
    #new_df = file.append(df, verify_integrity=True)
    final_df.to_csv('Weather_Data.csv', mode="a", header=False) # action: retain same index
else:
    final_df.to_csv('Weather_Data.csv')
end=time.time()
print(end-start)


C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_inde

C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_inde

C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_inde

C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_inde

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3280 entries, 2009-01-01 to 2018-10-28
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Average temperature (°F)  3280 non-null   object
 1   Average humidity (%)      3280 non-null   object
 2   Average dewpoint (°F)     3280 non-null   object
 3   Average barometer (in)    3280 non-null   object
 4   Average windspeed (mph)   3280 non-null   object
 5   Average gustspeed (mph)   3280 non-null   object
 6   Average direction (°deg)  3280 non-null   object
 7   Rainfall for month (in)   3280 non-null   object
 8   Rainfall for year (in)    3280 non-null   object
 9   Maximum rain per minute   3280 non-null   object
 10  Maximum temperature (°F)  3280 non-null   object
 11  Minimum temperature (°F)  3280 non-null   object
 12  Maximum humidity (%)      3280 non-null   object
 13  Minimum humidity (%)      3280 non-null   object
 14  Maximu

C:\Users\mehul\AppData\Local\Temp\ipykernel_124224\1493142063.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df=final_df.append(df, ignore_index=True)
